In [2]:
import xml.etree.ElementTree as ET
import json
# 解析XML文件
"""
转化下数据
"""
def build_dataset(dataset, split):
    tree = ET.parse(dataset + '/' + split + "_" + dataset + '.xml')
    root = tree.getroot()
    data = []
    sentiment = {"positive":0,"neutral":0,"negative":0,"conflict":0}
    for i in root.findall('sentence'):
        single_data = {'text': i.find('text').text, "id": i.get('id')}
        data.append(single_data)
        if i.find('aspectTerms'):
            for e in i.find('aspectTerms').findall('aspectTerm'):
                term = e.get('term')
                polarity = e.get('polarity') # negative/positive/neutral/conflict
                implicit = e.get('implicit_sentiment', "####") # False/True
                opinion = e.get('opinion_words', '####') # False/True
                start = e.get('from')
                end = e.get('to')
                if "aspects" in single_data:
                    single_data['aspects'].append({'term': term, "polarity": polarity, "implicit": implicit, "from_to": [start, end], "opinion": opinion})
                else:
                    single_data['aspects'] = [{'term': term, "polarity": polarity, "implicit": implicit, "from_to": [start, end], "opinion": opinion}]
        if single_data not in data:
            data.append(single_data)
    tmp_data = []
    for i in data:
        if 'aspects' in i:
            text = i['text']
            for j in i['aspects']:
                user_prompt = "Analyze the sentiment polarity of aspects “{}” in the comment “{}”".format(j['term'], text)
                tmp_data.append({"question":user_prompt, "answer":j['polarity'], "original_data":i})
                sentiment[j['polarity']] = sentiment[j['polarity']] + 1
    print(len(tmp_data))
    print(sentiment)
    with open(dataset+"_"+split+".json", "w+", encoding='utf-8') as f:
        json.dump(tmp_data, f, ensure_ascii=False, indent=4)

datasets = ["laptops", "restaurants"]
splits = ["train", "test"]
# datasets = ["laptops"]
# splits = ["train"]
for d in datasets:
    for s in splits:
        build_dataset(d, s)

2358
{'positive': 987, 'neutral': 460, 'negative': 866, 'conflict': 45}
654
{'positive': 341, 'neutral': 169, 'negative': 128, 'conflict': 16}
3693
{'positive': 2164, 'neutral': 633, 'negative': 805, 'conflict': 91}
1134
{'positive': 728, 'neutral': 196, 'negative': 196, 'conflict': 14}


In [3]:
import json
with open("laptops_train.json", "r+", encoding='utf-8') as f:
    train_data = json.load(f)
with open("laptops_test.json", "r+", encoding='utf-8') as f:
    test_data = json.load(f)
with open("laptops_train_test_extra.json", "w+", encoding='utf-8') as f:
    for i in test_data:
        if i['answer'] == "neutral" or i["answer"] == "conflict":
            train_data.append(i)
    print(len(train_data))
    json.dump(train_data, f, ensure_ascii=False, indent=4)

2543


In [1]:
a = [1,2,3]
b = [4,5,6]
a+b

[1, 2, 3, 4, 5, 6]

In [25]:
system_prompt = """You are a sentiment analysis assistant, performing sentiment analysis on a given laptop product review and corresponding aspects. There are only four emotion categories: positive, negative, conflict, and neutral. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <reasoning> </reasoning> and <answer> </answer> tags, respectively, i.e., <reasoning> reasoning process here </reasoning> <answer> answer here </answer>. For example, analyze the sentiment polarity of 4 aspects “battery, screen, price and warranty” in sentence "Good battery, bad screen, the price is expensive but worth it, not sure about the warranty". <reasoning> Let’s analyze it step by step. Users need to analyze the sentiment polarity of words in four aspects. The sentence expresses distinct opinions about different aspects. The phrase “Good battery” clearly conveys a positive sentiment toward the battery, using the adjective “good” without any qualifiers. In contrast, “bad screen” directly reflects a negative sentiment about the screen, as “bad” is an unambiguous negative descriptor. The statement “the price is expensive but worth it” presents a conflicted view of the price: while “expensive” suggests a negative perception, “worth it” introduces a compensating positive judgment, indicating mixed feelings. Finally, “not sure about the warranty” reveals uncertainty rather than any clear opinion, making the sentiment toward the warranty neutral. </reasoning> <answer> positive, negative, conflict, neutral </answer>"""

system_prompt = """You are a sentiment analysis assistant. Given a laptop product review and a list of aspects, your task is to determine the sentiment polarity for each aspect. The possible sentiment categories are: positive, negative, conflict, and neutral.

You must first explain your reasoning and then provide the final sentiment labels. Wrap your explanation inside <reasoning> tags and the final output inside <answer> tags. The format must be:

<reasoning> your reasoning here </reasoning>
<answer> label1, label2, ... </answer>

Example:
Sentence: "Good battery, bad screen, the price is expensive but worth it, not sure about the warranty"
Aspects: battery, screen, price, warranty
<reasoning>Let’s analyze it step by step. Users need to analyze the sentiment polarity of words in 4 aspects. The phrase “Good battery” expresses a clearly positive opinion toward the battery. “bad screen” reflects a negative sentiment about the screen. “the price is expensive but worth it” indicates a mix of negative and positive cues, thus a conflicted sentiment. “not sure about the warranty” shows uncertainty, so the sentiment is neutral. </reasoning>
<answer>positive, negative, conflict, neutral</answer>"""

In [33]:
import re

text = """1235436<reasoning>Let’s analyze it step by step. Users need to analyze the sentiment polarity of words in 4 aspects. The phrase “Good battery” expresses a clearly positive opinion toward the battery. “bad screen” reflects a negative sentiment about the screen. “the price is expensive but worth it” indicates a mix of negative and positive cues, thus a conflicted sentiment. “not sure about the warranty” shows uncertainty, so the sentiment is neutral. </reasoning>
<answer>positive, negative, conflict, neutral</answer>547239875"""

# 正则模式
pattern = r"<reasoning>((?:(?!<reasoning>|</reasoning>).)*?)</reasoning>\n<answer>((?:(?!<answer>|</answer>).)*?)</answer>"

# 查找第一个匹配
match = re.search(pattern, text, re.DOTALL)

# 输出结果
if match:
    print("✅ 提取成功：\n")
    print(match.group())
else:
    print("❌ 没有找到符合格式的内容")

✅ 提取成功：

<reasoning>Let’s analyze it step by step. Users need to analyze the sentiment polarity of words in 4 aspects. The phrase “Good battery” expresses a clearly positive opinion toward the battery. “bad screen” reflects a negative sentiment about the screen. “the price is expensive but worth it” indicates a mix of negative and positive cues, thus a conflicted sentiment. “not sure about the warranty” shows uncertainty, so the sentiment is neutral. </reasoning>
<answer>positive, negative, conflict, neutral</answer>


In [36]:
None in ["a"]

False

In [ ]:
import json
# 打开文件并读取JSON数据
with open("laptops_test.json", 'r', encoding='utf-8') as f:
    data = json.load(f)


In [ ]:
请阅读以下评论，并结合给定的方面词和情感极性，进行逐个方面的情感分析，推理过程应自然流畅，体现出模型在理解语义后作出的判断，而非列出参考答案。

【评论】：
I am pleased with the fast log on, speedy WiFi connection and the long battery life (>6 hrs).

【方面词及其情感】：
log on：positive
WiFi connection: positive
battery life: positive

请你结合文本内容，对每个方面进行推理分析，写出你是如何判断出其情感极性的。推理可以适当引用评论中的具体句子，但不要照搬原文，要用自己的话组织语言，使推理过程看起来像是你自己思考后的结果。
推理过程必须用英文表达。
如果包含多个方面词，只需要在一段话中返回全部推理。
只需要返回推理过程，不需要返回任何解释和说明，以json格式返回。

【输出格式】：
{“reasoning”：reasoning}  -> {str:str}

In [5]:
import json
from collections import Counter
# 打开文件并读取JSON数据
max_len = 0
with open("laptops_train_test.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
for i in data:
    # if len(i["question"]) > max_len:
    #     max_len = len(i["question"])
    if len(i["question"]) == 540:
        print(i)
print(max_len)

{'question': "Analyze the sentiment polarity of aspects “Windows update” in the comment “In\xa0November my computer messed up entirely and wouldn't power on after intalling a Windows update, I had to have my HD flashed and lost EVERYTHING on it, including my school assignments and irriplaceable pictures that were only in digital format and several other things, when this update was installed for some reason I was unable to roll back the drivers and everything to an earlier working condition because when the update was installed it deleted my history.”", 'answer': 'neutral', 'original_data': {'text': "In\xa0November my computer messed up entirely and wouldn't power on after intalling a Windows update, I had to have my HD flashed and lost EVERYTHING on it, including my school assignments and irriplaceable pictures that were only in digital format and several other things, when this update was installed for some reason I was unable to roll back the drivers and everything to an earlier wor

In [9]:
import json
with open("laptops/laptops_train.json", 'r', encoding='utf-8') as f:
    train_data = json.load(f)
with open("laptops/laptops_test.json", 'r', encoding='utf-8') as f:
    test_data = json.load(f)
data = test_data
for i in data:
    text = i["text"]
    if "aspects" in i:
        for j in i["aspects"]:
            if j['polarity'] == "neutral":
                print(text)
                print("aspect:  " + j['term'])
                print("sentiment polarity:  " + j['polarity'])
                print(' ')

No installation disk (DVD) is included.
aspect:  installation disk (DVD)
sentiment polarity:  neutral
 
The Mountain Lion OS is not hard to figure out if you are familiar with Microsoft Windows.
aspect:  Microsoft Windows
sentiment polarity:  neutral
 
It's been time for a new laptop, and the only debate was which size of the Mac laptops, and whether to spring for the retina display.
aspect:  size
sentiment polarity:  neutral
 
It's been time for a new laptop, and the only debate was which size of the Mac laptops, and whether to spring for the retina display.
aspect:  retina display
sentiment polarity:  neutral
 
Apple removed the DVD drive Firewire port (will work with adapter) and put the SDXC slot in a silly position on the back.
aspect:  DVD drive Firewire port
sentiment polarity:  neutral
 
Apple removed the DVD drive Firewire port (will work with adapter) and put the SDXC slot in a silly position on the back.
aspect:  adapter
sentiment polarity:  neutral
 
It is really thick arou

In [ ]:
[
  {
    "sentence": "No fingerprint reader included, but I rarely need it anyway.",
    "aspect": "fingerprint reader",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The charging cable is fragile, but at least it charges fast.",
    "aspect": "charging cable",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The hinges feel weak, though the screen stays stable during use.",
    "aspect": "hinges",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The touchpad is unresponsive at times, but multi-touch gestures still work well.",
    "aspect": "touchpad",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The speakers aren't very loud, but the sound quality is pretty clear.",
    "aspect": "speakers",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "It lacks an SD card slot, but I use an external adapter so it's fine.",
    "aspect": "SD card slot",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The fan gets noisy under heavy load, but at least it keeps the system cool.",
    "aspect": "fan",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The resolution could be higher, but colors and sharpness are decent.",
    "aspect": "resolution",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The trackpad takes getting used to, but gestures are smooth once you adapt.",
    "aspect": "trackpad",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "Startup time is a bit long, but once it's on, performance is great.",
    "aspect": "startup time",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "Build quality could be better, but the design is sleek and functional.",
    "aspect": "build quality",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "No USB-C port, but adapters are cheap and easy to use.",
    "aspect": "USB-C port",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The display has a yellowish tint, but it's still very readable.",
    "aspect": "display",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "Keyboard backlight isn't very bright, though it's enough for dim environments.",
    "aspect": "keyboard backlight",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "Battery drains quickly during gaming, but lasts long in normal use.",
    "aspect": "battery",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "It's bulkier than most laptops, but I love the solid build.",
    "aspect": "size",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The webcam is low resolution, but it’s okay for quick calls.",
    "aspect": "webcam",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "It took a while to set up dual boot, but it works perfectly now.",
    "aspect": "dual boot",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "Doesn't support 5GHz WiFi, but 2.4GHz works fine in most places.",
    "aspect": "WiFi",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "Keyboard is a little cramped, though typing feels responsive.",
    "aspect": "keyboard",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The brightness isn’t very strong outdoors, but looks sharp indoors.",
    "aspect": "brightness",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The hard drive space is limited, but cloud syncing solves that.",
    "aspect": "hard drive",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "No physical volume buttons, but the shortcuts are easy to use.",
    "aspect": "volume buttons",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "Build feels plasticky, but it's surprisingly durable.",
    "aspect": "material",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The display refresh rate is low, yet good enough for everyday work.",
    "aspect": "refresh rate",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "No optical drive, but who uses DVDs anymore?",
    "aspect": "optical drive",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "Cooling system is loud, but the laptop never gets too hot.",
    "aspect": "cooling system",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "Color accuracy isn't perfect, though editing is still manageable.",
    "aspect": "color accuracy",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "It’s heavier than expected, but that’s due to its rugged chassis.",
    "aspect": "weight",
    "sentiment polarity": "conflict"
  },
  {
    "sentence": "The system feels dated in parts, yet surprisingly snappy overall.",
    "aspect": "system performance",
    "sentiment polarity": "conflict"
  }
]

In [50]:
import json
import csv

with open("laptops_train.json", "r+", encoding="utf-8") as f:
    train_data = json.load(f)
# neutral_data = []
# with open("dataaugmentation/500——neutral_test.csv", "r+", encoding="utf-8") as f:
#     csv_reader = csv.reader(f)
#     next(csv_reader)
#     for i in csv_reader:
#         neutral_data.append({
#             "question": "Analyze the sentiment polarity of aspects “{}” in the comment “{}”".format(i[1], i[0].strip('"')),
#             "answer": i[2]})
# print(len(list(set(neutral_data))))
# conflict_data = []
# with open("dataaugmentation/___700_conflict__test.csv", "r+", encoding="utf-8") as f:
#     csv_reader = csv.reader(f)
#     next(csv_reader)
#     for i in csv_reader:
#         conflict_data.append(str({
#             "question": "Analyze the sentiment polarity of aspects “{}” in the comment “{}”".format(i[1], i[0].strip('"')),
#             "answer": i[2]}))
# print(len(list(set(conflict_data))))

In [40]:
# 为生成 700 条风格多样、不重复的 conflict 样本，采用更大规模模板库 + 多样 aspect 组合
import  random
# 构造更丰富的冲突类表达模板（含正负并存情感）
diverse_conflict_templates = list(set([
    "The {aspect} disappointed me at first, but I've grown to appreciate it.",
    "Not completely happy with the {aspect}, yet it still works out fine.",
    "The {aspect} has issues, though I find ways to work around them.",
    "Although the {aspect} lacks in some areas, overall it's usable.",
    "The {aspect} is annoying at times, but still gets the job done.",
    "I don't love the {aspect}, but I wouldn't call it a problem either.",
    "Some flaws in the {aspect}, but performance is still acceptable.",
    "The {aspect} is a mixed bag — both good and bad.",
    "There’s room for improvement in the {aspect}, though it’s not unusable.",
    "Though the {aspect} can be frustrating, I’ve adjusted to it.",
    "The {aspect} isn’t ideal, but far from terrible.",
    "While I expected better from the {aspect}, it's fine for now.",
    "There's definitely something off about the {aspect}, but I can live with it.",
    "The {aspect} doesn't meet my expectations, though it’s passable.",
    "Mixed feelings about the {aspect} — it helps and hinders.",
    "The {aspect} underwhelmed me at first, but now it’s tolerable.",
    "The {aspect} gave me trouble at first, but it's manageable with use.",
    "I'm not satisfied with the {aspect}, though it's not a deal-breaker.",
    "The {aspect} isn't consistent, yet still functional enough.",
    "The {aspect} falls short sometimes, but it serves the purpose.",
    "Honestly, the {aspect} could be better, but it's still usable day to day.",
    "The {aspect} leaves something to be desired, yet it doesn’t affect my experience too much.",
    "While not perfect, the {aspect} manages to meet my basic expectations.",
    "I've had a few problems with the {aspect}, but nothing major.",
    "The {aspect} doesn’t always behave as expected, though I’ve found workarounds.",
    "The {aspect} isn’t the smoothest, but I’ve come to live with it.",
    "Even with its flaws, the {aspect} is functional enough for my needs.",
    "Despite some complaints, the {aspect} hasn’t been a real obstacle.",
    "Sometimes the {aspect} acts up, but I can’t say it ruins the experience.",
    "The {aspect} is far from flawless, although it performs decently overall.",
    "I had doubts about the {aspect}, but it's turned out okay in practice.",
    "There are glitches with the {aspect}, but nothing show-stopping.",
    "The {aspect} can be inconvenient, yet not a deal-breaker.",
    "Although I’ve faced minor issues with the {aspect}, it’s mostly reliable.",
    "The {aspect} takes some patience, but it's ultimately serviceable.",
    "You might notice imperfections in the {aspect}, though they don’t matter much.",
    "While occasionally frustrating, the {aspect} works most of the time.",
    "The {aspect} feels outdated, but still serves its role.",
    "I was expecting more from the {aspect}, but it hasn’t held me back too much.",
    "Minor problems aside, the {aspect} functions reasonably well."
]))

# 增加更广泛的方面词
conflict_aspects = list(set([
    "price", "screen", "keyboard", "battery life", "weight", "trackpad", "sound", "RAM", "camera",
    "build quality", "fan", "wifi", "bluetooth", "charger", "materials", "graphics", "heat", "noise",
    "speed", "startup time", "sleep mode", "USB ports", "touchscreen", "software", "display", "design",
    "resolution", "cooling system", "performance", "size", "port layout", "webcam", "function keys",
    "case", "setup process", "durability", "stability", "OS", "power adapter", "speakers", "features",
    "audio", "charging port", "brightness", "login speed", "boot time", "firmware", "trackpoint",
    "fan noise", "loudness", "typing experience", "build", "color", "LED indicator", "hard drive"
]))

# 使用集合记录生成过的组合以确保不重复
unique_conflict_sentences = set()
conflict_samples_final = []

# 开始生成直到达到 700 条不重复样本
while len(conflict_samples_final) < 900:
    aspect = random.choice(conflict_aspects)
    template = random.choice(diverse_conflict_templates)
    sentence = template.format(aspect=aspect)
    if sentence in unique_conflict_sentences:
        continue
    unique_conflict_sentences.add(sentence)
    conflict_samples_final.append(str({
            "question": "Analyze the sentiment polarity of aspects “{}” in the comment “{}”".format(aspect, sentence.strip('"')),
            "answer": "conflict"}))
conflict_samples_final = list(set(conflict_samples_final))

In [39]:
import random
import json

# 模板句式库（中性情感，不包含正负评价）
templates = [
    "I just installed the {aspect} but haven’t really explored its use yet.",
    "The {aspect} is included in the package, but I haven’t used it so far.",
    "Still learning how to make use of the {aspect}, but nothing special yet.",
    "The device comes with a built-in {aspect}, but I haven’t tested it.",
    "This model supports {aspect}, although I haven't found a reason to try it.",
    "There’s a {aspect} on the side, but I rarely interact with it.",
    "I noticed the {aspect} during setup, but didn’t pay much attention.",
    "I'm aware of the {aspect}, but it hasn't impacted my experience either way.",
    "The laptop features a {aspect}, though I haven't explored it fully.",
    "I connected the {aspect} once just to check, and it worked fine.",
    "So far I haven’t had to use the {aspect}.",
    "Not sure how often I’ll use the {aspect}, but it’s there.",
    "My setup includes a {aspect}, though I don’t rely on it.",
    "I haven’t figured out what the {aspect} does yet.",
    "The {aspect} is functional, but I haven't really had a need for it.",
    "It came with a {aspect}, which I left untouched.",
    "The presence of {aspect} was noted but hasn’t been relevant yet.",
    "I saw the {aspect} option, but skipped it during configuration.",
    "The {aspect} is part of the hardware, but I don’t actively use it.",
    "I believe it has a {aspect}, though I haven’t confirmed that myself.",
    "I haven't interacted with the {aspect} since setting it up.",
    "There's a {aspect} I haven’t figured out yet.",
    "I don’t know if the {aspect} matters for how I use the laptop.",
    "Maybe I’ll try the {aspect} sometime, but I don’t see the need right now.",
]

# 构造一个随机aspect词表（避免固定）
aspect_list = [
    "HDMI port", "Ethernet port", "Bluetooth", "fingerprint reader", "keyboard backlight",
    "VGA port", "USB-C connector", "external monitor", "microphone jack", "SD card slot",
    "trackpad gestures", "volume control", "system preferences", "login screen", "fan control",
    "speaker grille", "light sensor", "air vent", "SIM card tray", "camera shutter",
    "RAM", "SSD", "BIOS", "Thunderbolt port", "recovery partition",
    "power brick", "retina display", "webcam", "screen resolution", "language settings",
    "voice assistant", "IR sensor", "startup chime", "pre-installed apps", "macOS dock",
    "time machine", "disk utility", "migration assistant", "system report", "activity monitor",
    "file system", "energy saver mode", "keyboard layout", "screen mirroring", "dual boot option",
    "network diagnostics", "mission control", "login items", "USB 3.0 port", "touch ID"
]

# 随机生成数据
neutral_data = []
for _ in range(800):
    aspect = random.choice(aspect_list)
    template = random.choice(templates)
    sentence = template.format(aspect=aspect)
    neutral_data.append(str({
            "question": "Analyze the sentiment polarity of aspects “{}” in the comment “{}”".format(aspect, sentence.strip('"')),
            "answer": "neutral"}))

neutral_data = list(set(neutral_data))


In [41]:
neutral_data = [eval(i) for i in neutral_data]
conflict_data = [eval(i) for i in conflict_samples_final]

In [43]:
print(len(neutral_data))
print(len(conflict_data))

587
900


In [44]:
neutral_data[0]

{'question': "Analyze the sentiment polarity of aspects “keyboard backlight” in the comment “The laptop features a keyboard backlight, though I haven't explored it fully.”",
 'answer': 'neutral'}

In [52]:
train_data += conflict_data + neutral_data

In [58]:
train_data = [eval(j) for j in list(set([str(i) for i in train_data]))]

In [69]:
len(train_data)

3811

In [66]:
train_data[1000]["answer"]

'positive'

In [59]:
with open("dataaugmentation/augmentation_laptops_train.json", "w+", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

In [68]:
from collections import Counter
with open("dataaugmentation/augmentation_laptops_train.json", "r+", encoding="utf-8") as f:
    data = json.load(f)
senti = []
for i in data:
    senti.append(i["answer"])
print(Counter(senti))

Counter({'neutral': 1043, 'positive': 976, 'conflict': 945, 'negative': 847})
